# 流路の可視化


In [1]:
import struct
import numpy as np
import os
import re

In [2]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [3]:
from src import OpenFile

In [4]:
echo %PATH%

c:\Users\yucca\00_WorcSpace\Valid-Error-Tpfit\venv\Scripts;C:\Users\yucca\00_WorcSpace\Valid-Error-Tpfit\venv\Scripts;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files\Git\cmd;C:\Users\yucca\AppData\Local\Microsoft\WindowsApps;C:\Users\yucca\AppData\Local\Programs\Microsoft VS Code\bin;C:\Users\yucca\AppData\Local\Programs\Python\Python311\Scripts;


In [6]:
#入力データのディレクトリ
data_path=r"input"
# 現在のカレントディレクトリを取得します
directory_path = os.getcwd()
directory_path

'c:\\Users\\yucca\\00_WorcSpace\\Valid-Error-Tpfit\\notebooks'

In [8]:
t=OpenFile.make_folder("output",directory_path)
print("output_path:", t)
#shared_name,data_files=OpenFile.get_shared_name(data_path)
shared_name="case60g02out"

shared_name

Folder 'output' already exists in 'c:\Users\yucca\00_WorcSpace\Valid-Error-Tpfit\notebooks'.
output_path: c:\Users\yucca\00_WorcSpace\Valid-Error-Tpfit\notebooks\output


'case60g02out'

In [9]:
struct_name=shared_name+"S_00000"
struct_path=os.path.join(data_path,struct_name)
struct_path

'input\\case60g02outS_00000'

In [18]:
filename_s=f"input\{shared_name}S_00000"

In [27]:
data_s_origin=np.array(OpenFile.get_data_from_binary(filename_s)) 
data_s_origin

array([0., 0., 0., ..., 0., 0., 0.])

In [39]:
#座標データ抽出
points = []
for d in ["X","Y","Z"]:

    data_path_ndim=f"{data_path}/{shared_name}{d}_00000"
    #ファイル読み込み
    with open(data_path_ndim, 'rb') as file:
        # バイナリデータを読み取り
        binary_data_ndim = file.read()
  
    # バイナリデータを文字列に変換（適切なエンコーディングを指定）
    text_data = binary_data_ndim.decode('utf-8')  # 例としてUTF-8を使用

    # 正規表現を使って「X-Axis Coordinate」に関連するデータを抽出
    pattern_d= re.compile(r'{}-Axis Coordinate\n([\s\S]*?)(?=\n[A-Z]|$)'.format(d))
    match_d = pattern_d.search(text_data)
    if match_d:
        coordinates = match_d.group(1).strip().split()
        # 数値を浮動小数点数に変換
        coordinates = np.array([float(coord) for coord in coordinates] )
        points.append(coordinates)

In [40]:
x_points=points[0][1:-1]
y_points=points[1][1:-1]
z_points=points[2][1:-1]
x_points.shape,y_points.shape,z_points.shape

((60,), (60,), (472,))

In [41]:
x_crop=points[0][[0,-1]]
y_crop=points[1][[0,-1]]
z_crop=points[2][[0,-1]]
x_crop,y_crop,z_crop

(array([-0.001525,  0.001525]),
 array([-0.001525,  0.001525]),
 array([-2.5000e-05,  2.0025e-02]))

In [42]:
data_shape=(points[0].shape[0],points[1].shape[0],points[2].shape[0])
data_shape

(62, 62, 474)

In [43]:
data_s = data_s_origin.reshape(data_shape, order="F")[1:-1,1:-1, 1:-1]
data_s.shape

(60, 60, 472)

In [69]:
plot_points=[ (0,1,2), (30,30,10), (30,30,400)]

plot_c = [ 1,5, 300]

In [71]:
x,y,z = np.meshgrid(points[0],points[1],points[2], indexing='ij')
print(z.shape)
for i, index in enumerate(plot_points):
    print(index)
    print(plot_c[i],(x[index],y[index],z[index]))

(62, 62, 474)
(0, 1, 2)
1 (np.float64(-0.001525), np.float64(-0.001475), np.float64(7.5e-05))
(30, 30, 10)
5 (np.float64(-2.5e-05), np.float64(-2.5e-05), np.float64(0.000475))
(30, 30, 400)
300 (np.float64(-2.5e-05), np.float64(-2.5e-05), np.float64(0.016375))


In [72]:
# 流路の描画
# 座標点のプロット